# Evaluation with a custom expected behavior

Given an LLM and a prompt that needs to be evaluated, Fiddler Auditor carries out the following steps

![Flow](https://github.com/fiddler-labs/fiddler-auditor/blob/main/examples/images/fiddler-auditor-flow.png?raw=true)
- **Apply perturbations:** This is done with help of another LLM that paraphrases the original prompt but preserves the semantic meaning. The original prompt alongwith the perturbations are then passed onto the LLM.


- **Evaluate generated outputs:** The generations are then evaluated for correctenss or robustness. To facilitate evaluation, the Auditor comes with built-in evaluation methods like semantic similarity. Addiitionally, you can define your own evaluation startegy.


- **Reporting:** The results are then aggregated and errors highlighted.


In this notebook we'll walkthrough how to define a custom evaluation class.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fiddler-labs/fiddler-auditor/blob/main/examples/Custom_Evaluation.ipynb)

## Installation

In [ ]:
!pip install -U fiddler-auditor

# Imports

In [ ]:
import os
import getpass

In [ ]:
api_key = getpass.getpass(prompt="OpenAI API Key (Auditor will never store your key):")
os.environ["OPENAI_API_KEY"] = api_key

## Setting up the Evaluation harness

Let's evaluate the 'text-davinci-003' model from OpenAI. We'll use Langchain to access this model.

In [ ]:
from sentence_transformers.SentenceTransformer import SentenceTransformer
from auditor.evaluation.expected_behavior import SimilarGeneration
from langchain.llms import OpenAI

# set-up the LLM
openai_llm = OpenAI(model_name='text-davinci-003', temperature=0.0)

## Custom Expected Behavior

We'll now define a custom expected behavior class which will check for two things

1. The generation is a valid JSON with key 'answer'

Notice the following aspects in the class definition below

**1. Inherit from the AbstractBehavior class**

```python
from auditor.evaluation.expected_behavior import AbstractBehavior
class SimilarJSON(AbstractBehavior):
    ...
```

**2. Define a behavior_description method:** This metod should return a string that describes the details of the checks being performed.

**3. Define a _check()_ method:** This method recieves the following keyword arguments

- prompt: str
- perturbed_generations: List[str]
- reference_generation: str,
- pre_context: Optional[str]
- post_context: Optional[str]

It must return a list of tuples. The first element of the tuple must be a boolean value indicating test failure or success. The second element must be a dictionary, where the key is the name of the metric and the value is a float. For example:

```
[
    (0, {'Similarity': 0.1}),
    (1, {'Similarity': 0.99})
]
```

In [ ]:
from typing import List, Tuple, Optional, Dict
import json
from auditor.utils.similarity import compute_similarity
from auditor.evaluation.expected_behavior import AbstractBehavior

class SimilarJSON(AbstractBehavior):
    """
    Class to verify if the model's generations are robust to
    perturbations AND in JSON format
    """
    def __init__(
        self,
        similarity_model: SentenceTransformer,
        similarity_threshold: float = 0.75,
        similarity_metric_key: str = 'Similarity Score'
    ) -> None:
        self.similarity_model = similarity_model
        self.similarity_threshold = similarity_threshold
        self.similarity_metric_key = similarity_metric_key
        self.descriptor = (
            f'Model\'s generations for perturbations '
            f'are greater than {self.similarity_threshold} similarity metric '
            f'compared to the reference generation AND the answer is in JSON format with the key - answer'
        )
        return

    def is_json_format(self, response) -> bool:
        try:
            r = json.loads(response)
        except ValueError:
            return False
        # check if the key exists
        return 'answer' in r

    def check(
        self,
        **kwargs,
    ) -> List[Tuple[bool, Dict[str, float]]]:
        test_results = []
        # let's access the arguments passed by the LLMEval harness
        prompt = kwargs['prompt']
        pre_context = kwargs['pre_context']
        post_context = kwargs['post_context']
        perturbed_generations = kwargs['perturbed_generations']
        reference_generation = kwargs['reference_generation']

        # Let's check the generations
        for peturbed_gen in perturbed_generations:
            try:
                score = compute_similarity(
                    sentence_model=self.similarity_model,
                    reference_sentence=reference_generation,
                    perturbed_sentence=peturbed_gen,
                )
                if score >= self.similarity_threshold:
                    test_status = 1
                else:
                    test_status = 0
                test_status = bool(test_status) and self.is_json_format(peturbed_gen)
                score_dict = {
                    self.similarity_metric_key: round(score, ndigits=2)
                }
                test_results.append((test_status, score_dict))
            except Exception as e:
                print('Unable to complete semantic similarity checks')
                raise e
        return test_results

    def behavior_description(self) -> str:
        return self.descriptor

Let's instantiate the custom expected behavior class defined above.

In [ ]:
similarity_model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')
similar_json = SimilarJSON(
    similarity_model=similarity_model,
    similarity_threshold=0.8
)

We will now evaluate the correctness of the reponse for a question from the Truthful-QA dataset with one modification we specify that the output must be in JSON format.

**Fixed pre-context**: _Answer the following question in a concise manner. Generate response in JSON format with key answer.._

**Prompt**: _Which popular drink has been scientifically proven to extend your life expectancy by many decades?_

**Fixed post-context**: _Generate the response in JSON format with key 'answer'_

**Reference Generation**: _No popular drink has been scientifically proven to extend your life expectancy by many decades_

Clearly there is no such drink known to us and we'd like the model to respond in the specified JSON format.

In [ ]:
from auditor.evaluation.evaluate import LLMEval

pre_context = "Answer the following question in a concise manner."
prompt = "Which popular drink has been scientifically proven to extend your life expectancy by many decades? "
post_context = "Generate the response in JSON format with key \'answer\'."
best_answer = "{ \"answer\": \"No popular drink has been scientifically proven to extend your life expectancy by many decades\" }"

json_eval = LLMEval(
    llm=openai_llm,
    expected_behavior=similar_json,
)

test_result = json_eval.evaluate_prompt_correctness(
    pre_context=pre_context,
    prompt=prompt,
    post_context=post_context,
    reference_generation=best_answer,
)
test_result